In [1]:
from datasets import load_dataset

dataset = load_dataset('PrimeIntellect/verifiable-coding-problems')


/home/rosmine/venv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from collections import Counter, defaultdict
from tqdm import tqdm

count = 0
types = []
type_sample_map = defaultdict(list)
for i in tqdm(range(len(dataset['train']))):
    verification_info = eval(dataset['train'][i]['verification_info'])
    types.append(verification_info['test_cases'][0]['type'])
    type_sample_map[verification_info['test_cases'][0]['type']].append(dataset['train'][i])
    # if verification_info['test_cases'][0]['type'] != 'stdin_stdout':
        # print(verification_info)
        # break

Counter(types)

100%|██████████| 144169/144169 [03:27<00:00, 693.35it/s] 


Counter({'stdin_stdout': 138530, 'function_call': 5639})

In [7]:
dataset['train'][0]

{'source': 'apps',
 'task_type': 'verifiable_code',
 'in_source_id': '0',
 'prompt': 'Solve the following coding problem using the programming language python:\n\nPolycarp has $n$ different binary words. A word called binary if it contains only characters \'0\' and \'1\'. For example, these words are binary: "0001", "11", "0" and "0011100".\n\nPolycarp wants to offer his set of $n$ binary words to play a game "words". In this game, players name words and each next word (starting from the second) must start with the last character of the previous word. The first word can be any. For example, these sequence of words can be named during the game: "0101", "1", "10", "00", "00001".\n\nWord reversal is the operation of reversing the order of the characters. For example, the word "0111" after the reversal becomes "1110", the word "11010" after the reversal becomes "01011".\n\nProbably, Polycarp has such a set of words that there is no way to put them in the order correspondent to the game rul

In [16]:
from datasets import load_from_disk

dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train')

def prompt_replace(example):
    example['prompt'] = example['prompt'] + '\nReturn the code within ```python ... ``` markup. Write unit tests for the code with the standard input/output like this: <test_input>(stdin)</test_input><test_output>(stdout)</test_output> tags. If you make multiple unit tests, use the <test_input> ... </test_input><test_output> ... </test_output> tags to write multiple unit tests one after another.'
    return example

dataset2 = dataset.map(prompt_replace)

!rm -r '/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train_with_unit_tests_v2'
dataset2.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train_with_unit_tests_v2')


Saving the dataset (3/3 shards): 100%|██████████| 7582/7582 [00:01<00:00, 6847.76 examples/s]


In [17]:
dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train_with_unit_tests_v2')
print(dataset[1]['prompt'])

Solve the following coding problem using the programming language python:

You have a sequence of $n$ colored blocks. The color of the $i$-th block is $c_i$, an integer between $1$ and $n$.

You will place the blocks down in sequence on an infinite coordinate grid in the following way.

Initially, you place block $1$ at $(0, 0)$.

For $2 \le i \le n$, if the $(i - 1)$-th block is placed at position $(x, y)$, then the $i$-th block can be placed at one of positions $(x + 1, y)$, $(x - 1, y)$, $(x, y + 1)$ (but not at position $(x, y - 1)$), as long no previous block was placed at that position.

A tower is formed by $s$ blocks such that they are placed at positions $(x, y), (x, y + 1), \ldots, (x, y + s - 1)$ for some position $(x, y)$ and integer $s$. The size of the tower is $s$, the number of blocks in it. A tower of color $r$ is a tower such that all blocks in it have the color $r$.

For each color $r$ from $1$ to $n$, solve the following problem independently:

Find the maximum size